<a href="https://colab.research.google.com/github/VGN-SK/iBot_DC_app/blob/main/EP25B041_PH1050_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eigh
from scipy.integrate import trapezoid
from ipywidgets import interact, FloatSlider, IntSlider

hbar = 6.582119569e-16       #Scaling all units in terms of eV and nm
m = 5.68563e6                # eV·s²/nm² - here the particle considered is electron
eV = 1.0

def build_hamiltonian(x, V):
    N = len(x)
    dx = x[1] - x[0]
    diag = np.ones(N) * -2.0
    off_diag = np.ones(N - 1) * 1.0
    T = (-hbar**2 / (2 * m * dx**2)) * (
        np.diag(diag) + np.diag(off_diag, 1) + np.diag(off_diag, -1)
    )
    return T + np.diag(V)  # This is the Hamiltonian matrix

def infinite_square_well(L=1.0, N=800, num_states=4):
    x = np.linspace(0, L, N + 2)[1:-1]
    V = np.zeros(N)
    H = build_hamiltonian(x, V)
    E_si, psi = eigh(H)
    E = E_si
    psi = psi / np.sqrt(trapezoid((psi)**2, x, axis=0))

    n_vals = np.arange(1, num_states + 1)
    E_analytical = (n_vals**2 * (hbar**2 * np.pi**2) / (2 * m * L**2))
    psi_analytical = np.array([np.sqrt(2 / L) * np.sin(n * np.pi * x / L) for n in n_vals]).T

    for n in range(num_states):
      if np.sign(psi[1, n]) != np.sign(psi_analytical[1, n]): # This is because both + or - ψ is valid and eigh function picks randomly .
        psi[:, n] *= -1                                       # This is done to correct it .

    plt.figure(figsize=(10, 6)) #plotting probabiliy density
    for n in range(num_states):
        plt.plot(x, psi[:, n]**2, label=f"Numerical ψ² n={n+1}")
        plt.plot(x, psi_analytical[:, n]**2, '--', label=f"Analytical ψ² n={n+1}")
    plt.title(f"Infinite Square Well (L={L} nm)")
    plt.xlabel("x (nm)")
    plt.ylabel("Probability density")
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(10, 6)) # plotting wave function
    for n in range(num_states):
        plt.plot(x, psi[:, n], label=f"Numerical ψ (n={n+1})")
        plt.plot(x, psi_analytical[:, n], '--', label=f"Analytical ψ (n={n+1})")
    plt.title(f"Infinite Square Well (L={L} nm)")
    plt.xlabel("x (nm)")
    plt.ylabel("Wave function ψ ")
    plt.legend()
    plt.grid(True)
    plt.show()

    print("Energy levels comparison (eV):")
    print(f"{'n':>2} {'Numerical (eV)':>20} {'Analytical (eV)':>20}")  # Comparing the eigenvalues obtained to the analytical values of energy
    for n in range(num_states):
        print(f"{n+1:2d} {E[n]:20.10e} {E_analytical[n]:20.10e}")


def step_potential(L=1.0, a=0.5, V0=50.0, N=800, num_states=4):
    x = np.linspace(0, L, N)
    V = np.zeros(N)
    V[x > a] = V0
    H = build_hamiltonian(x, V)
    E_si, psi = eigh(H)
    E = E_si
    psi = psi / np.sqrt(trapezoid((psi)**2, x, axis=0)) # Normalization of values

    plt.figure(figsize=(9, 5)) #plotting probability density
    plt.plot(x, V, 'k--', label='V(x)')
    for n in range(num_states):
        plt.plot(x, psi[:, n]**2, label=f"ψ², n={n+1}")
    plt.title(f"Finite Step Potential: V₀={V0} eV, a={a} nm")
    plt.xlabel("x (nm)")
    plt.ylabel("Probability density")
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(9, 5)) # plotting wave functions
    plt.plot(x, V, 'k--', label='V(x)')
    for n in range(num_states):
        plt.plot(x, psi[:, n], label=f"ψ, n={n+1}")
    plt.title(f"Finite Step Potential: V₀={V0} eV, a={a} nm")
    plt.xlabel("x (nm)")
    plt.ylabel("Wave function ψ")
    plt.legend()
    plt.grid(True)
    plt.show()

    print("Energy levels (eV):")
    for n in range(num_states):
        print(f"n={n+1}: E = {E[n]:.10e}")

def interactive_step(V0=20.0, a=0.5, L=1.0):
    step_potential(L=L, a=a, V0=V0, N=800, num_states=4)

def interactive_well(L=1.0):
    infinite_square_well(L=L, N=800, num_states=4)

interact(
    interactive_well,
    L=FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Well Width L (nm)'),
)

interact(
    interactive_step,
    V0=FloatSlider(value=15, min=0.0, max=50.0, step=0.1, description='Barrier Height V₀ (eV)'),
    a=FloatSlider(value=0.5, min=0.2, max=0.8, step=0.05, description='Step Position a (nm)'),
    L=FloatSlider(value=1.0, min=0.8, max=2.0, step=0.1, description='Box Width L (nm)'),
)

interactive(children=(FloatSlider(value=1.0, description='Well Width L (nm)', max=2.0, min=0.5), Output()), _d…

interactive(children=(FloatSlider(value=15.0, description='Barrier Height V₀ (eV)', max=50.0), FloatSlider(val…

<function __main__.interactive_step(V0=20.0, a=0.5, L=1.0)>